# Importing Overture data from Parquet

The [Overture foundation/project](https://overturemaps.org/download/) has developed a novel dataset that serves as a supplement to OpenStreetMap. There is a lot more to it, so please check out the link above.

This notebook will highlight how to parse their open data into GeoPandas Dataframes

In [1]:
import sys, os, importlib
import rasterio, geojson

import pandas as pd
import geopandas as gpd

from sedona.spark import * 

%load_ext autoreload
%autoreload 2

/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [9]:
config = SedonaContext.builder().config("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider").getOrCreate()
sedona = SedonaContext.create(config)



Py4JJavaError: An error occurred while calling o52.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task serialization failed: org.apache.spark.SparkException: Failed to register classes with Kryo
org.apache.spark.SparkException: Failed to register classes with Kryo
	at org.apache.spark.serializer.KryoSerializer.$anonfun$newKryo$5(KryoSerializer.scala:185)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.Utils$.withContextClassLoader(Utils.scala:240)
	at org.apache.spark.serializer.KryoSerializer.newKryo(KryoSerializer.scala:173)
	at org.apache.spark.serializer.KryoSerializer$$anon$1.create(KryoSerializer.scala:104)
	at com.esotericsoftware.kryo.pool.KryoPoolQueueImpl.borrow(KryoPoolQueueImpl.java:48)
	at org.apache.spark.serializer.KryoSerializer$PoolWrapper.borrow(KryoSerializer.scala:111)
	at org.apache.spark.serializer.KryoSerializerInstance.borrowKryo(KryoSerializer.scala:351)
	at org.apache.spark.seria

In [ ]:
df = sedona.read.format("parquet").load("s3a://overturemaps-us-west-2/release/2023-07-26-alpha.0/theme=buildings/type=building")

In [6]:
tempDF = gpd.read_parquet("s3://overturemaps-us-west-2/release/2023-07-26-alpha.0/theme=buildings/type=building")

OSError: When getting information for key 'release/2023-07-26-alpha.0/theme=buildings/type=building' in bucket 'overturemaps-us-west-2': AWS Error [code 15]: No response body.